In [70]:
import os
import pandas as pd
import numpy as np
import pathlib
import xarray as xr
import rioxarray


In [71]:
import geopandas as gpd

In [72]:
read_file_path: str = r"./../../data/zmax_center.json"
"""读取文件"""
data = gpd.read_file(read_file_path)
data

,最大淹没深度(cm),geometry
0,2,"POLYGON ((121.5443 29.1733, 121.5449 29.1733, ..."
1,8,"POLYGON ((121.5449 29.1733, 121.5455 29.1733, ..."
2,1,"POLYGON ((121.5437 29.1727, 121.5443 29.1727, ..."
3,12,"POLYGON ((121.5443 29.1727, 121.5449 29.1727, ..."
4,7,"POLYGON ((121.5455 29.1727, 121.5461 29.1727, ..."
...,...,...
16951,11,"POLYGON ((121.6175 28.8415, 121.6181 28.8415, ..."
16952,8,"POLYGON ((121.6181 28.8415, 121.6187 28.8415, ..."
16953,90,"POLYGON ((121.6187 28.8415, 121.6193 28.8415, ..."
16954,3,"POLYGON ((121.6181 28.8409, 121.6187 28.8409, ..."


In [73]:
data.head

<bound method NDFrame.head of        最大淹没深度(cm)                                           geometry
0               2  POLYGON ((121.5443 29.1733, 121.5449 29.1733, ...
1               8  POLYGON ((121.5449 29.1733, 121.5455 29.1733, ...
2               1  POLYGON ((121.5437 29.1727, 121.5443 29.1727, ...
3              12  POLYGON ((121.5443 29.1727, 121.5449 29.1727, ...
4               7  POLYGON ((121.5455 29.1727, 121.5461 29.1727, ...
...           ...                                                ...
16951          11  POLYGON ((121.6175 28.8415, 121.6181 28.8415, ...
16952           8  POLYGON ((121.6181 28.8415, 121.6187 28.8415, ...
16953          90  POLYGON ((121.6187 28.8415, 121.6193 28.8415, ...
16954           3  POLYGON ((121.6181 28.8409, 121.6187 28.8409, ...
16955          89  POLYGON ((121.6187 28.8409, 121.6193 28.8409, ...

[16956 rows x 2 columns]>

In [74]:
data.iloc[0]

最大淹没深度(cm)                                                    2
geometry      POLYGON ((121.5443 29.1733, 121.5449 29.1733, ...
Name: 0, dtype: object

In [75]:
var_name='最大淹没深度(cm)'

In [76]:
surge_values = data[var_name].values

In [77]:
surge_values

array([ 2,  8,  1, ..., 90,  3, 89], dtype=int32)

In [78]:
len(surge_values)

16956

In [79]:
# 提取多边形的几何中心作为坐标
# 这里我们使用多边形的质心（centroid）作为代表坐标
centroids = data.geometry.centroid
latitudes = centroids.y
longitudes = centroids.x

# 创建一个xarray.Dataset
ds = xr.Dataset(
    {
        "surge": (("feature"), surge_values)
    },
    coords={
        "feature": np.arange(len(data)),
        "latitude": ("feature", latitudes),
        "longitude": ("feature", longitudes)
    }
)

C:\Users\evase\AppData\Local\Temp\ipykernel_135672\546687521.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = data.geometry.centroid


In [80]:
ds

<xarray.Dataset> Size: 475kB
Dimensions:    (feature: 16956)
Coordinates:
  * feature    (feature) int64 136kB 0 1 2 3 4 ... 16951 16952 16953 16954 16955
    latitude   (feature) float64 136kB 29.17 29.17 29.17 ... 28.84 28.84 28.84
    longitude  (feature) float64 136kB 121.5 121.5 121.5 ... 121.6 121.6 121.6
Data variables:
    surge      (feature) int32 68kB 2 8 1 12 7 6 12 12 ... 3 90 90 11 8 90 3 89

In [81]:
out_put_path:str='./../../data/zmax.nc'

In [82]:
ds.to_netcdf(out_put_path)

In [83]:
from shapely.geometry import box

# 定义网格长宽
L = 0.0005  # 例如，0.001度的网格

# 获取整个区域的边界
minx, miny, maxx, maxy = data.total_bounds

# 生成网格
x_coords = np.arange(minx, maxx, L)
y_coords = np.arange(miny, maxy, L)

In [84]:
data.total_bounds

array([121.3931,  28.8403, 121.7585,  29.1733])

In [85]:
# 创建一个空的二维数组来存储网格数据
grid_data = np.full((len(y_coords), len(x_coords)), np.nan)

In [86]:
grid_data

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [87]:
# 遍历每个多边形，将其属性值映射到网格
for _, row in data.iterrows():
    poly = row.geometry
    surge_value = row[var_name]
    
    # 找到多边形覆盖的网格索引
    for i, x in enumerate(x_coords):
        for j, y in enumerate(y_coords):
            cell = box(x, y, x + L, y + L)
            if poly.intersects(cell):
                grid_data[j, i] = surge_value


KeyboardInterrupt: 

In [ ]:
# 创建xarray.Dataset
ds = xr.Dataset(
    {
        "surge": (("y", "x"), grid_data)
    },
    coords={
        "x": x_coords,
        "y": y_coords
    }
)


In [ ]:

# 将Dataset保存为NetCDF文件
ds.to_netcdf('output.nc')

-----

#### 优化效率

In [ ]:
import geopandas as gpd
import xarray as xr
import numpy as np
from shapely.geometry import box

# 读取GeoJSON文件
gdf = gpd.read_file(read_file_path)

# 定义网格长宽
L = L  # 例如，0.001度的网格

# 获取整个区域的边界
minx, miny, maxx, maxy = gdf.total_bounds

# 生成网格
x_coords = np.arange(minx, maxx, L)
y_coords = np.arange(miny, maxy, L)

# 创建一个空的二维数组来存储网格数据
grid_data = np.full((len(y_coords), len(x_coords)), np.nan)

# 创建空间索引
spatial_index = gdf.sindex

# 遍历网格
for i, x in enumerate(x_coords):
    for j, y in enumerate(y_coords):
        cell = box(x, y, x + L, y + L)
        
        # 使用空间索引查找可能相交的多边形
        possible_matches_index = list(spatial_index.intersection(cell.bounds))
        possible_matches = gdf.iloc[possible_matches_index]
        
        # 检查实际相交的多边形
        for _, row in possible_matches.iterrows():
            if row.geometry.intersects(cell):
                grid_data[j, i] = row[var_name]
                break  # 一旦找到一个相交的多边形，就可以退出循环

# 创建xarray.Dataset
ds = xr.Dataset(
    {
        "surge": (("y", "x"), grid_data)
    },
    coords={
        "x": x_coords,
        "y": y_coords
    }
)



In [ ]:
ds

<xarray.Dataset> Size: 4MB
Dimensions:  (y: 667, x: 731)
Coordinates:
  * x        (x) float64 6kB 121.4 121.4 121.4 121.4 ... 121.8 121.8 121.8 121.8
  * y        (y) float64 5kB 28.84 28.84 28.84 28.84 ... 29.17 29.17 29.17 29.17
Data variables:
    surge    (y, x) float64 4MB nan nan nan nan nan nan ... nan nan nan nan nan

In [ ]:
out_put_path:str='./../../data/zmax.nc'
ds.to_netcdf(out_put_path)

#### step2: 将dataset 按照小于1.0，小于1.5进行筛选，并存储为geotiff

In [ ]:
ds['surge'] = ds['surge'].where(ds['surge'] > 100, np.nan)

In [ ]:
ds.to_netcdf('./../../data/zmax_lte_100.nc')

In [ ]:
import numpy as np
import xarray as xr
import rasterio
from rasterio.transform import from_origin

# 假设 ds 是之前创建并过滤后的 xarray.Dataset

# 获取坐标和数据
surge_data = ds['surge'].values
x_coords = ds['x'].values
y_coords = ds['y'].values

# 定义仿射变换
transform = from_origin(min(x_coords), max(y_coords), x_coords[1] - x_coords[0], y_coords[1] - y_coords[0])

# 定义元数据
meta = {
    'driver': 'GTiff',
    'dtype': 'float32',
    'nodata': np.nan,
    'width': len(x_coords),
    'height': len(y_coords),
    'count': 1,  # 单波段
    'crs': 'EPSG:4326',  # 假设使用WGS84坐标系
    'transform': transform
}

# 反转 y 轴的数据，因为 GeoTIFF 的原点在左上角
surge_data_flipped = np.flipud(surge_data)

# 写入 GeoTIFF 文件
with rasterio.open('./../../data/zmax_lte_100.tif', 'w', **meta) as dst:
    dst.write(surge_data_flipped, 1)

#### step3: 将过滤后的dataset提取边界为多边形并存储为geojson  
以下方法有误

In [ ]:
import numpy as np
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape, mapping
import json

# 假设 surge_data_flipped 是你的数据
# 例如，surge_data_flipped = np.flipud(surge_data)

# 创建一个掩码，标识非 NaN 的区域
mask = ~np.isnan(surge_data_flipped)

# 打开一个虚拟的内存文件，模拟 rasterio 的操作
with rasterio.io.MemoryFile() as memfile:
    with memfile.open(
        driver='GTiff',
        height=surge_data_flipped.shape[0],
        width=surge_data_flipped.shape[1],
        count=1,
        dtype=surge_data_flipped.dtype,
        transform=from_origin(0, 0, 1, 1)  # 假设单元格大小为 1x1，原点在 (0,0)
    ) as dataset:
        dataset.write(surge_data_flipped, 1)

        # 将栅格数据转换为矢量多边形
        results = (
            {'properties': {'raster_val': v}, 'geometry': s}
            for i, (s, v) 
            in enumerate(shapes(surge_data_flipped, mask=mask, transform=dataset.transform))
        )

        # 转换为 GeoJSON 格式
        geojson = {
            "type": "FeatureCollection",
            "features": list(results)
        }

# 保存为 GeoJSON 文件
with open('./../../data/zmax_lte_100.json', 'w') as f:
    json.dump(geojson, f)

c:\Users\evase\anaconda3\envs\geopandas_env\Lib\site-packages\rasterio\io.py:143: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  rd = writer(


In [ ]:
ds

<xarray.Dataset> Size: 4MB
Dimensions:  (y: 667, x: 731)
Coordinates:
  * x        (x) float64 6kB 121.4 121.4 121.4 121.4 ... 121.8 121.8 121.8 121.8
  * y        (y) float64 5kB 28.84 28.84 28.84 28.84 ... 29.17 29.17 29.17 29.17
Data variables:
    surge    (y, x) float64 4MB nan nan nan nan nan nan ... nan nan nan nan nan

In [ ]:
data_array=ds['surge'].values

In [ ]:
data_array

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [ ]:
lon=ds.coords['x']
lat=ds.coords['y']

In [ ]:
lon_max=max(lon.values)
lon_min=min(lon.values)
lat_max=max(lat.values)
lat_min=min(lat.values)


In [ ]:
len(lon)

731

step 3-2：生成边界并提取为多边形

In [ ]:
from shapely.ops import unary_union

In [88]:
data_array_T=data_array.T

In [ ]:
# 创建掩码，标识非 NaN 的区域
mask = ~np.isnan(data_array_T)

# 使用 rasterio 和 shapes 提取多边形
transform = rasterio.transform.from_bounds(lon_min, lat_min, lon_max, lat_max, len(lon), len(lat))
transform


Affine(np.float64(0.0004993160054743404), np.float64(0.0), np.float64(121.3931),
       np.float64(0.0), np.float64(-0.0004992503748114302), np.float64(29.173299999999223))

In [95]:

with rasterio.io.MemoryFile() as memfile:
    with memfile.open(
        driver='GTiff',
        height=data_array_T.shape[0],
        width=data_array_T.shape[1],
        count=1,
        dtype=data_array_T.dtype,
        crs='EPSG:4326',  # 使用 WGS84 坐标系
        transform=transform
    ) as dataset:
        dataset.write(data_array_T, 1)

        # 将栅格数据转换为矢量多边形
        results = (
            {'properties': {'raster_val': v}, 'geometry': s}
            for i, (s, v) 
            in enumerate(shapes(data_array_T, mask=mask, transform=dataset.transform))
        )

        # 提取所有多边形
        polygons = [shape(feature['geometry']) for feature in results]

        # 合并多边形
        merged_polygon = unary_union(polygons)

ValueError: Mask must have same shape as image

In [91]:
# 准备 GeoJSON 数据
geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": mapping(merged_polygon)
        }
    ]
}

# 保存为 GeoJSON 文件
with open('./../../data/zmax_lte_100_1114_T.json', 'w') as f:
    json.dump(geojson, f)

# print("GeoJSON 文件已生成：output_polygon.geojson")

#### step3-3: 直接读取本地geotiff文件 此种方法有问题


In [69]:
# 读取 GeoTIFF 文件
file_path = './../../data/zmax_lte_100.tif'  # 替换为你的 GeoTIFF 文件路径

with rasterio.open(file_path) as src:
    # 读取第一个波段
    band = src.read(1)
    
    # 创建掩码，标识非 NaN 的区域
    mask = band != src.nodata

    # 提取多边形
    results = (
        {'properties': {'surge': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(shapes(band, mask=mask, transform=src.transform))
    )

    # 提取所有多边形
    polygons = [shape(feature['geometry']) for feature in results]

    # 合并多边形
    merged_polygon = unary_union(polygons)

# 准备 GeoJSON 数据
geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": mapping(merged_polygon)
        }
    ]
}

# 保存为 GeoJSON 文件
output_geojson_path = 'output_polygon.geojson'
with open(output_geojson_path, 'w') as f:
    json.dump(geojson, f)
